In [1]:
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install rouge_score sacrebleu textstat evaluate datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 31.7 MB/s eta 0:00:00:00:0100:01
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-czpj12yz/unsloth_8b1d9ed78a584883aaf5063a88f5ef48
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-czpj12yz/unsloth_8b1d9ed78a584883aaf5063a88f5ef48
  Resolved https://github.com/unslothai/unsloth.git to commit d1e312dcdc57bf020aa0f6da810226efe79cd69a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import time, csv, uuid, datetime, types
import torch
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
import evaluate

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-05 18:41:33.606857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764960094.002901      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764960094.105877      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[xformers|WARNING]WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.9 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.9 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Config

class Config:
    MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    MAX_SEQ_LENGTH = 2048
    DTYPE = None
    LOAD_IN_4BIT = True
    DATASET_PATH = "/kaggle/input/bengali-empathetic-conversations-corpus/BengaliEmpatheticConversationsCorpus .csv"
    OUTPUT_DIR = "llama3.1-bengali-finetuned"
    PER_DEVICE_BATCH_SIZE = 2
    GRADIENT_ACCUMULATION_STEPS = 1
    MAX_STEPS = 30
    LEARNING_RATE = 2e-4
    SEED = 3407
    LORA_R = 16
    LORA_ALPHA = 16
    LORA_DROPOUT = 0.0

In [4]:
# Loggers

class ExperimentLogger:
    def __init__(self, filename="LLAMAExperiments.csv"):
        self.filename = filename
        self.headers = ["id", "model_name", "lora_config", "train_loss", "val_loss", "metrics", "timestamp"]
        if not os.path.exists(self.filename):
            with open(self.filename, 'w', newline='', encoding='utf-8') as f:
                csv.writer(f).writerow(self.headers)
    def log_experiment(self, exp_data):
        with open(self.filename, 'a', newline='', encoding='utf-8') as f:
            csv.writer(f).writerow(exp_data)

class ResponseLogger:
    def __init__(self, filename="GeneratedResponses.csv"):
        self.filename = filename
        self.headers = ["id", "experiment_id", "input_text", "response_text", "timestamp"]
        if not os.path.exists(self.filename):
            with open(self.filename, 'w', newline='', encoding='utf-8') as f:
                csv.writer(f).writerow(self.headers)
    def log_response(self, exp_id, input_text, response_text):
        with open(self.filename, 'a', newline='', encoding='utf-8') as f:
            csv.writer(f).writerow([str(uuid.uuid4()), exp_id, input_text, response_text, datetime.datetime.now().isoformat()])


In [5]:
# Dataset Preprocessor

class DatasetProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
    def load_data(self):
        if not os.path.exists(self.file_path):
            local = os.path.basename(self.file_path)
            if os.path.exists(local):
                self.file_path = local
        try:
            df = pd.read_csv(self.file_path, encoding='utf-8')
        except Exception:
            df = pd.read_csv(self.file_path, encoding='latin-1')
        return Dataset.from_pandas(df)
    def format_prompt_func(self, examples):
        qs = examples["Questions"]
        as_ = examples["Answers"]
        texts = []
        for q, a in zip(qs, as_):
            txt = (
                "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
                f"{q}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
                f"{a}<|eot_id|>"
            )
            texts.append(txt)
        return {"text": texts}

In [6]:
# Strategy

class UnslothStrategy:
    def setup_model(self):
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=Config.MODEL_ID,
            max_seq_length=Config.MAX_SEQ_LENGTH,
            dtype=Config.DTYPE,
            load_in_4bit=Config.LOAD_IN_4BIT,
            device_map="auto"
        )
        model = FastLanguageModel.get_peft_model(
            model,
            r=Config.LORA_R,
            target_modules=[
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj"
            ],
            lora_alpha=Config.LORA_ALPHA,
            lora_dropout=Config.LORA_DROPOUT,
            bias="none",
            use_gradient_checkpointing="unsloth",
            random_state=Config.SEED,
        )
        return model, tokenizer

In [7]:
# Fine-tuner

class LLAMAFineTuner:
    def __init__(self, strategy):
        self.strategy = strategy
        self.model = None
        self.tokenizer = None
        self.trainer = None
    def initialize(self):
        self.model, self.tokenizer = self.strategy.setup_model()
        print("MODEL DEVICE INFO (if available):")
        try:
            print(self.model.hf_device_map if hasattr(self.model, "hf_device_map") else "no hf_device_map attribute")
        except Exception:
            pass
        print("CUDA devices:", torch.cuda.device_count(), " Current device:", torch.cuda.current_device() if torch.cuda.is_available() else "cpu")
    def train(self, train_dataset):
        print("Starting Training...")
        training_args = TrainingArguments(
            per_device_train_batch_size=Config.PER_DEVICE_BATCH_SIZE,
            gradient_accumulation_steps=Config.GRADIENT_ACCUMULATION_STEPS,
            warmup_steps=5,
            max_steps=Config.MAX_STEPS,
            learning_rate=Config.LEARNING_RATE,
            fp16=True,
            bf16=False,
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=Config.SEED,
            output_dir=Config.OUTPUT_DIR,
            report_to="none",
            dataloader_pin_memory=True,
        )
        self.trainer = SFTTrainer(
            model=self.model,
            tokenizer=self.tokenizer,
            train_dataset=train_dataset,
            dataset_text_field="text",
            max_seq_length=Config.MAX_SEQ_LENGTH,
            dataset_num_proc=1,
            packing=False,
            args=training_args,
        )

        def _compute_loss_and_wrap(self_trainer, model, inputs, return_outputs=False, *args, **kwargs):
            outputs = model(**inputs)
            if hasattr(outputs, "loss"):
                loss = outputs.loss
            elif isinstance(outputs, (tuple, list)) and len(outputs) > 0:
                loss = outputs[0]
            else:
                loss = outputs
            if not isinstance(loss, torch.Tensor):
                try:
                    device = next(model.parameters()).device
                except StopIteration:
                    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                loss = torch.tensor(float(loss), dtype=torch.float32, device=device)
            if return_outputs:
                return loss, outputs
            return loss

        self.trainer.compute_loss = types.MethodType(_compute_loss_and_wrap, self.trainer)

        return self.trainer.train()

In [8]:
# Evaluator

class Evaluator:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        try:
            self.bleu = evaluate.load("bleu")
            self.rouge = evaluate.load("rouge")
        except Exception:
            self.bleu = None
            self.rouge = None
        self.response_logger = ResponseLogger()
        self.exp_id = int(time.time())
    def evaluate(self, test_data, num_samples=5):
        try:
            FastLanguageModel.for_inference(self.model)
        except Exception:
            pass
        predictions, references = [], []
        limit = min(num_samples, len(test_data))
        print(f"\n--- Generating {limit} responses ---")
        for i in range(limit):
            q = test_data[i]["Questions"]
            a = test_data[i]["Answers"]
            prompt = (
                "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
                f"{q}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
            )
            inputs = self.tokenizer([prompt], return_tensors="pt", truncation=True, padding=True, max_length=Config.MAX_SEQ_LENGTH)
            try:
                inputs = {k: v.to("cuda") for k, v in inputs.items()}
            except Exception:
                pass
            outputs = self.model.generate(**inputs, max_new_tokens=128, use_cache=True)
            generated_text = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            response = generated_text.split("assistant")[-1].strip()
            predictions.append(response)
            references.append(a)
            self.response_logger.log_response(self.exp_id, q, response)
            print(f"\nQ: {q}\n-> {response}\n")
        results = {}
        try:
            if self.bleu:
                results["BLEU"] = self.bleu.compute(predictions=predictions, references=references)
            if self.rouge:
                results["ROUGE"] = self.rouge.compute(predictions=predictions, references=references)
        except Exception as e:
            print("Metric error:", e)
        return results


In [9]:
# Main

if __name__ == "__main__":
    torch.manual_seed(Config.SEED)
    logger = ExperimentLogger()
    csv_path = Config.DATASET_PATH
    processor = DatasetProcessor(csv_path)
    dataset = processor.load_data()
    formatted_dataset = dataset.map(processor.format_prompt_func, batched=True, remove_columns=dataset.column_names)
    strategy = UnslothStrategy()
    tuner = LLAMAFineTuner(strategy)
    tuner.initialize()

    # Tokenize & set format to torch
    def tokenize_for_training(batch):
        out = tuner.tokenizer(batch["text"], truncation=True, padding="max_length", max_length=Config.MAX_SEQ_LENGTH)
        out["labels"] = out["input_ids"].copy()
        return out

    formatted_dataset = formatted_dataset.map(tokenize_for_training, batched=True, remove_columns=formatted_dataset.column_names)
    formatted_dataset.set_format(type="torch")

    # Train
    stats = tuner.train(formatted_dataset)
    print("Training Complete!")

    # Evaluate
    evaluator = Evaluator(tuner.model, tuner.tokenizer)
    metrics = evaluator.evaluate(dataset, num_samples=5)

    # Log
    train_loss = getattr(stats, "training_loss", None)
    logger.log_experiment([
        int(time.time()),
        "Llama-3.1-8B-Unsloth",
        f"r={Config.LORA_R}, alpha={Config.LORA_ALPHA}",
        train_loss,
        "N/A",
        str(metrics),
        datetime.datetime.now().isoformat()
    ])

    try:
        tuner.model.save_pretrained("lora_model")
        print("Saved lora_model.")
    except Exception as e:
        print("Save failed:", e)

    print("\n DONE.")


Map:   0%|          | 0/38233 [00:00<?, ? examples/s]

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 38,233 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,18.397100
2,18.110000
3,17.889700
4,18.070500
5,17.216700
6,16.997300
7,15.508000
8,12.879600
9,10.822700
10,10.400600


Training Complete!



--- Generating 5 responses ---

Q:  আমার স্ত্রী এবং মায়ের মধ্যে টানটান মতবিরোধ চলছে। অতীতে, তাদের মধ্যে ছোটখাটো পার্থক্য ছিল। উদাহরণস্বরূপ, আমার স্ত্রী আমার কাছে অভিযোগ করবে যে আমার মা খুব কর্তৃত্বপ্রয়াসী; আমার মা অভিযোগ করবেন আমার স্ত্রী অলস। তবে ইদানীং তা তীব্রতর হয়েছে । আমি মনে করি, এর কারণ হচ্ছে আমার স্ত্রী তার সাথে একবার কথার প্রতিত্তর করেছিল। এখন, যেকোনো সামান্য মতবিরোধকে বড় করা হয়, যা বড় মতবিরোধের দিকে পরিচালিত করে। আমি কি করতে পারি?
-> এটা স্বাভাবিক যে আপনার স্ত্রী এবং আপনার মা একেকের সাথে বিভিন্নভাবে আচরণ করছেন। এটা আমাদের সাধারণ দৃষ্টিভঙ্গি �


Q:  আমি বাচ্চা নেওয়ার পরিকল্পনা করছি, তাই আমাকে ধূমপান ছেড়ে দিতে হবে - কিন্তু এটা কঠিন। কখনও কখনও এটি শারীরিক প্রয়োজন নয়, এটি মানসিক। আমি ধূমপান সম্পর্কে চিন্তা করা থেকে নিজেকে বিরত করতে পারি না। এই আসক্তি থেকে মুক্তি পেতে আমি কি করতে পারি?
-> আপনার ধূমপান থেকে নিজেকে বিরত করার জন্য একটি পরিকল্পনা বাস্তবায়নের জন্য কয়েকটি পদক্ষেপ নিম্নরূপ:

1.  **পরি�


Q:  আমার মনের মধ্যে গোপন আছে, এবং আমি জানি না তাদের সঙ্গে কি করতে হবে. 